## Orth potential

In [ ]:
import sympy as sp
from sympy.utilities.codegen import codegen
def gen_c(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="C",
    )[0][1]

# 定义符号
a11, a12, a21, a22 = sp.symbols('a11 a12 a21 a22', real=True)

# 定义矩阵 a
a = sp.Matrix([[a11, a12], [a21, a22]])

# 定义单位矩阵
eye = sp.Matrix.eye(2)

# 计算正交性误差的 Frobenius 范数
orth = (a @ a.T - eye).norm(2)  # 使用 Frobenius 范数

# 对向量 [a11, a12, a21, a22] 求导
grad_orth = sp.Matrix([
    sp.diff(orth, a11),
    sp.diff(orth, a12),
    sp.diff(orth, a21),
    sp.diff(orth, a22)
])

# 计算 Hessian 矩阵
hess_orth = sp.Matrix([
    [sp.diff(grad_orth[0], a11), sp.diff(grad_orth[0], a12), sp.diff(grad_orth[0], a21), sp.diff(grad_orth[0], a22)],
    [sp.diff(grad_orth[1], a11), sp.diff(grad_orth[1], a12), sp.diff(grad_orth[1], a21), sp.diff(grad_orth[1], a22)],
    [sp.diff(grad_orth[2], a11), sp.diff(grad_orth[2], a12), sp.diff(grad_orth[2], a21), sp.diff(grad_orth[2], a22)],
    [sp.diff(grad_orth[3], a11), sp.diff(grad_orth[3], a12), sp.diff(grad_orth[3], a21), sp.diff(grad_orth[3], a22)]
])

print("Orthogonality Error:", orth)
print("Gradient:", grad_orth)
print("Hessian:", hess_orth)


In [ ]:
hess_orth[0, 0]

In [ ]:
for i in range(4):
    for j in range(4):
        # name = f"hess_orth_{i}{j}"
        # ccode = gen_c(name, hess_orth[i, j])
        # path = f'ccode/{name}.c'
        path = f'src/sim/generated/abd/hess/hess_orth_{i}{j}.rs'
        with open(path, 'w') as file:
            file.write("// This code is generated. \n")
        print(f'Saved {path}')

In [ ]:

print(gen_c("hess_orth_00", grad_orth[0, 0]))

## Contact

In [ ]:
import sympy as sp

# 定义符号
px, py = sp.symbols('px py', real=True)
x1x, x1y = sp.symbols('x1x x1y', real=True)
x2x, x2y = sp.symbols('x2x x2y', real=True)
a11, a12, a21, a22 = sp.symbols('a11 a12 a21 a22', real=True)
tx, ty = sp.symbols('tx ty', real=True)

# 定义点 p 和向量 x1, x2
p = sp.Matrix([px, py])
x1 = sp.Matrix([x1x, x1y])
x2 = sp.Matrix([x2x, x2y])

# 定义矩阵 a 和向量 t
a = sp.Matrix([[a11, a12], [a21, a22]])
t = sp.Matrix([tx, ty])

# 定义顶点 e1 和 e2
e1 = a @ x1 + t
e2 = a @ x2 + t


## Case 1

In [ ]:
# 计算向量 ap
ap = p - e1

# 计算距离
distance_case1 = ap.norm()

# 对 [px, py, a11, a12, a21, a22, tx, ty] 求导
derivatives_case1 = sp.Matrix([
    sp.diff(distance_case1, tx),
    sp.diff(distance_case1, ty),
    sp.diff(distance_case1, a11),
    sp.diff(distance_case1, a12),
    sp.diff(distance_case1, a21),
    sp.diff(distance_case1, a22),
])



print("Case 1 Distance:", distance_case1)
print("Case 1 Derivatives:", derivatives_case1)
d1_simp = sp.simplify(derivatives_case1)


In [ ]:
from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
    )[0][1]

ccode = gen_rust("d_grad_case1", derivatives_case1)

print(ccode)

## Case 2

In [ ]:
# 计算向量 bp
bp = p - e2

# 计算距离
distance_case2 = bp.norm()

# 对 [px, py, a11, a12, a21, a22, tx, ty] 求导
derivatives_case2 = sp.Matrix([
    sp.diff(distance_case2, px),
    sp.diff(distance_case2, py),
    sp.diff(distance_case2, a11),
    sp.diff(distance_case2, a12),
    sp.diff(distance_case2, a21),
    sp.diff(distance_case2, a22),
    sp.diff(distance_case2, tx),
    sp.diff(distance_case2, ty),
])

print("Case 2 Distance:", distance_case2)
print("Case 2 Derivatives:", derivatives_case2)
d2_simp = sp.simplify(derivatives_case2)



In [ ]:
derivatives_case2

## Case 3

In [ ]:
# 计算向量 ab 和 ap
ab = e2 - e1
ap = p - e1

# 计算 ap 在 ab 上的投影
ap_proj_on_ab = (ap.dot(ab) / ab.norm()**2) * ab

# 计算投影点
proj_point = e1 + ap_proj_on_ab

# 计算距离
distance_case3 = (p - proj_point).norm()

# 对 [px, py, a11, a12, a21, a22, tx, ty] 求导
derivatives_case3 = sp.Matrix([
    sp.diff(distance_case3, px),
    sp.diff(distance_case3, py),
    sp.diff(distance_case3, tx),
    sp.diff(distance_case3, ty),
    sp.diff(distance_case3, a11),
    sp.diff(distance_case3, a12),
    sp.diff(distance_case3, a21),
    sp.diff(distance_case3, a22),

])

print("Case 3 Distance:", distance_case3)
print("Case 3 Derivatives:", derivatives_case3)


In [ ]:
q = sp.Matrix([tx,ty,a11,a12,a21,a22])

hess3 = sp.Matrix([
    [distance_case3.diff(q[i]).diff(q[j]) for i in range(6)] for j in range(6)
])

In [ ]:
hess3